In [4]:
import os

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import pickle


with open('PIT_dict.pkl', 'rb') as f:
    PIT_dict = pickle.load(f)

PIT_dict = PIT_dict['complete_data']
PIT_dict

,Month,Day,Hour,Minute,PIT_DHI,PIT_DNI,PIT_GHI,PIT_Clearsky DHI,PIT_Clearsky DNI,PIT_Clearsky GHI,PIT_Dew Point,PIT_Surface Albedo,PIT_Wind_X,PIT_Wind_Y,PIT_Relative Humidity,PIT_Temperature,PIT_Pressure,PIT_Global Horizontal UV Irradiance (280-400nm),PIT_Global Horizontal UV Irradiance (295-385nm)
0,1,1,0,0,0,0,0,0,0,0,-14.0,0.866,-0.524772,-0.290886,81.30,-11.0,990,0.0,0.0
1,1,1,0,30,0,0,0,0,0,0,-14.0,0.866,-0.524772,-0.290886,81.32,-11.0,990,0.0,0.0
2,1,1,1,0,0,0,0,0,0,0,-14.0,0.866,-0.508829,-0.317952,79.56,-11.0,990,0.0,0.0
3,1,1,1,30,0,0,0,0,0,0,-14.0,0.866,-0.508829,-0.317952,79.57,-11.0,990,0.0,0.0
4,1,1,2,0,0,0,0,0,0,0,-14.0,0.866,-0.430146,-0.418299,78.91,-12.0,990,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385435,12,31,21,30,0,0,0,0,0,0,12.8,0.120,-0.363192,-0.712805,97.06,13.3,971,0.0,0.0
385436,12,31,22,0,0,0,0,0,0,0,13.3,0.120,-0.363192,-0.712805,98.46,13.5,970,0.0,0.0
385437,12,31,22,30,0,0,0,0,0,0,13.3,0.120,-0.423935,-0.678438,96.57,13.8,970,0.0,0.0
385438,12,31,23,0,0,0,0,0,0,0,13.9,0.120,-0.541634,-0.718772,98.45,14.1,970,0.0,0.0
